[View in Colaboratory](https://colab.research.google.com/github/Maximluking/AndroidWeather/blob/master/Copy_of_IMDB_Movie_Reviews_Classification.ipynb)

# **Загружаем модули**

In [0]:
from keras.datasets import imdb
from keras import models
from keras import layers

from keras.preprocessing.text import Tokenizer

import numpy as np


'''

https://github.com/keras-team

можно посчитать колличество слов в позитивных и негативнх рецензиях. скорее всего положительные длиннее

вывести сколько положительных и отрицательных отзывов в тренировочной и тестовой выборке
интересен баланс
ощущение что негативных намного больше

факты о базе данных

Dataset of 25,000 movies reviews from IMDB, labeled by sentiment (positive/negative). 
Reviews have been preprocessed, and each review is encoded as a sequence of word indexes (integers). 
For convenience, words are indexed by overall frequency in the dataset, 
so that for instance the integer "3" encodes the 3rd most frequent word in the data.
This allows for quick filtering operations such as: "only consider the top 10,000 most common words,
but eliminate the top 20 most common words".
As a convention, "0" does not stand for a specific word, but instead is used to encode any unknown word.
'''

Using TensorFlow backend.


'\n\nhttps://github.com/keras-team\n\nможно посчитать колличество слов в позитивных и негативнх рецензиях. скорее всего положительные длиннее\n\nвывести сколько положительных и отрицательных отзывов в тренировочной и тестовой выборке\nинтересен баланс\nощущение что негативных намного больше\n\nфакты о базе данных\n\nDataset of 25,000 movies reviews from IMDB, labeled by sentiment (positive/negative). \nReviews have been preprocessed, and each review is encoded as a sequence of word indexes (integers). \nFor convenience, words are indexed by overall frequency in the dataset, \nso that for instance the integer "3" encodes the 3rd most frequent word in the data.\nThis allows for quick filtering operations such as: "only consider the top 10,000 most common words,\nbut eliminate the top 20 most common words".\nAs a convention, "0" does not stand for a specific word, but instead is used to encode any unknown word.\n'

# edu_utils

In [0]:
import math

# прекращаем закодированные отзывы из цифр в текст для человека
def decode_review(word_index, review):
  reverse_word_index = dict(
  [(value, key) for (key, value) in word_index.items()])
  decoded_review = ' '.join(
  [reverse_word_index.get(i - 3, '?') for i in review])
  return decoded_review

# выводим отзыв для просмотра
def print_review(review):
  review_length = len(review)
  line_length = 150
  lines_number = math.ceil(review_length / float(line_length))
  for i in range(lines_number):
    if line_length > review_length:
      print(review)
    else:
      left_string_chank_len = review_length - i * line_length
      if left_string_chank_len < line_length:
        print(review[i * line_length:])
      else:
        print(review[i * line_length: (i + 1) * line_length])

# **Загружаем данные, делим их на валдицаию и тест, смотрим**

In [0]:
#1.Загружаем данные
unique_words = 10000
(train_data_imdb, train_labels_imdb), (test_data_imdb, test_labels_imdb) = imdb.load_data(num_words=unique_words)

validation_review_number = 1000
test_review_number = 1000

test_data   = test_data_imdb[:test_review_number]
test_labels = test_labels_imdb[:test_review_number] 

validation_data   = test_data_imdb[test_review_number : validation_review_number + test_review_number]
validation_labels = test_labels_imdb[test_review_number : validation_review_number + test_review_number]

train_data   = np.concatenate((train_data_imdb, test_data_imdb[validation_review_number + test_review_number:]), axis=0)
train_labels = np.concatenate((train_labels_imdb, test_labels_imdb[validation_review_number + test_review_number:]), axis=0)

# Загружаем словарь "из фир в слова"
word_index = imdb.get_word_index()

print('Колличество тренировачных обзоров: \n', train_data.shape[0],
      '\nКолличество валидационных обзоров: \n', validation_data.shape[0],
      '\nКолличество тестовых обзоров: \n', test_data.shape[0])

1646592/1641221 [==============================] - 1s 0us/step
Колличество тренировачных обзоров: 
 48000 
Колличество валидационных обзоров: 
 1000 
Колличество тестовых обзоров: 
 1000


In [0]:
# посмотреть один отзыв
review = decode_review(word_index, test_data[4])
print_review(review)

? like some other people wrote i'm a die hard mario fan and i loved this game br br this game starts slightly boring but trust me it's worth it as soo
n as you start your hooked the levels are fun and ? they will hook you ? your mind turns to ? i'm not kidding this game is also ? and is beautifully d
one br br to keep this spoiler free i have to keep my mouth shut about details but please try this game it'll be worth it br br story 9 9 action 10 1 
it's that good ? 10 attention ? 10 average 10


# **Обработка данных в нужнуый для сети формат**

In [0]:
#3.preprocess
tokenizer = Tokenizer(num_words=unique_words)
train_data_binary = tokenizer.sequences_to_matrix(train_data, mode='binary')
test_data_binary = tokenizer.sequences_to_matrix(test_data, mode='binary')
validation_data_binary = tokenizer.sequences_to_matrix(validation_data, mode='binary')

print('\nИсходный вид отзыва в базе данных: \n', train_data[1000],
      '\nОтзыв для человека: \n', review,
      '\nКак отзыв выглядит для нейронной сети: \n', train_data_binary)

print(train_data_binary.shape)
print(test_data_binary.shape)


Исходный вид отзыва в базе данных: 
 [1, 261, 13, 69, 110, 2, 11, 6, 750, 96, 145, 11, 2, 13, 426, 377, 233, 7, 4, 114, 549, 18, 3482, 1218, 7, 3199, 2102, 620, 5, 997, 429, 6, 4157, 7, 1654, 3404, 5, 1387, 2795, 4, 277, 10, 10, 103, 886, 49, 7, 4, 85, 857, 13, 1637, 56, 6, 1039, 7, 4, 4700, 626, 288, 8, 280, 174, 2541, 4, 182, 7, 2, 10, 10, 12, 505, 46, 14, 9, 31, 7, 148, 108, 1055, 315, 4, 7423, 15, 62, 140, 2555, 8, 374, 639, 4, 22, 381, 5810, 2, 3199, 2102, 17, 4409, 2, 2, 46, 7, 4, 1336, 8, 2, 4, 2952, 7, 2, 8, 2541, 5, 2363, 1176, 4, 500, 6, 2, 2, 2, 34, 4, 4698, 37, 9090, 27, 84, 34, 4578, 51, 934, 40, 2, 671, 4, 3359, 7, 4, 1973, 2290, 4, 323, 1650, 4824, 1510, 4409, 9, 4415, 11, 4, 1647, 1733, 34, 2549, 2, 37, 115, 2466, 42, 889, 4, 313, 280, 4415, 497, 8, 3890, 11, 19, 4, 5751, 34, 1658, 6, 1927, 767, 2, 19, 2, 7, 1336, 5, 428, 2964, 8, 135, 2, 659, 309, 620, 5, 997, 18, 27, 113, 367, 4, 1654, 1986, 7, 2, 23, 27, 96, 8, 6, 1304, 19, 4205, 5, 4, 500, 10, 10, 323, 3199, 2102, 

# **Строим модель**

In [0]:
#the model
model = models.Sequential()
model.add(layers.Dense(10, activation='relu', input_shape=(unique_words,)))
model.add(layers.Dropout(0.4))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                100010    
_________________________________________________________________
dropout_1 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                110       
_________________________________________________________________
dropout_2 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 100,131
Trainable params: 100,131
Non-trainable params: 0
_________________________________________________________________


# **Устанавливаем оптимизатор, функицю потерь, метрику**

In [0]:
model.compile(optimizer='rmsprop',
loss='binary_crossentropy',
metrics=['accuracy'])

# **Обучаем на тренировочны и проверяем на тестов данных**

In [0]:
print(train_labels.shape)
history = model.fit(train_data_binary, train_labels,
                    epochs=5,
                    batch_size=512,
                    validation_data=(validation_data_binary, validation_labels))

(48000,)
Train on 48000 samples, validate on 1000 samples
Epoch 1/5
48000/48000 [==============================] - 6s 130us/step - loss: 0.5057 - acc: 0.7726 - val_loss: 0.3544 - val_acc: 0.8840
Epoch 2/5
48000/48000 [==============================] - 4s 90us/step - loss: 0.3458 - acc: 0.8702 - val_loss: 0.2730 - val_acc: 0.8990
Epoch 3/5
48000/48000 [==============================] - 5s 97us/step - loss: 0.2788 - acc: 0.8999 - val_loss: 0.2462 - val_acc: 0.9040
Epoch 4/5
48000/48000 [==============================] - 5s 98us/step - loss: 0.2427 - acc: 0.9136 - val_loss: 0.2378 - val_acc: 0.9050
Epoch 5/5
48000/48000 [==============================] - 5s 97us/step - loss: 0.2177 - acc: 0.9229 - val_loss: 0.2399 - val_acc: 0.9040


# **Тестируем **

In [0]:
test_review = (test_data_binary[100:101])
y = model.predict(test_review)
print_review(test_review)
print('answer: ', y)

[[0. 1. 1. ... 0. 0. 0.]]
answer:  [[0.08259585]]


# **Перевод рецензий**

негатив 1. test_data[1000]

?? Это фильм был одним из худших, которые я когда либо видел ? Я смотрел его с моей девушкой на предыдущих выходных и продолжали его просмотр только лишь в надежде, что он станет лучше, но нет.
Качество самой картинки ужасное, такое ощущение , чтое его сняли на мобилку, плюс плохой свет не дает нормально проникнуться емоцияями на лицах актеров. Впрочем и сама игра актеров никуда не годится, если не брать во внимание, что это очередной тинейджерский ужастик. Звук был реальной проблемой: иногда приходилось ? звук в видео, потому что он был нечетким и не выдерживает никакой критики. Лучше не тратить на этот ? фильм своего времени.


? ? this is one of the worst movies i have ever seen the ? for the film is better than the film itself my girlfriend and i watched it this past weeken
d and we only continued to watch it in the hopes that it would get better it didn't br br the picture quality is poor it looks like it was shot on vid
eo and transferred to film the lighting is not great which makes it harder to read the actors' facial expressions the acting itself was cheesy but i g
uess it's acceptable for yet another teenage horror flick the sound was a huge problem sometimes you have to ? the video because the sound is unclear 
and or ? br br it holds no real merit of it's own trying to ride on the ? of sleepy hollow don't bother with this one

 негатив 2. test_data[4000]

? я был из тех избранных кто попал на ранний предпросмотр. Первое что вы должны знать об этом филье, что он основан на игре Far Cry, для своего 2004 года очень даже годная игра. Во-вторых вы должны знать, что за этим фильмом стоит человек по имени Уве Болл, который берет видео игру ? долгое время ? ? и тд и делает из них фильмы, очень ужасные должен я вам сказать. 
Я до сих пор помню как увидел ?  "Мечи короля" (The King's Swords ) в котором были ужасные ошибки,  к примеру показ 3 разных сцен одновременно причем две из них днем а третья волшебным образом ночью. Но все же вернусь к нашим пирогам. Если вы ожидаете от фильма Far Cry крутого экшена - забудте,  реально дешевые трюки и пластиковые вертолеты ассоциируются со всем кроме зрелищности, если у вас есть ожиданию по сценарию - забудте, даже изначальная легенда игры не спасла фильм от насмешек. Как ни странно игра актеров во многих моментах сделал фильм еще смешнее, только вот в плохом понимании этого слова. Ребята, у меня океазалась прекрасная возможность просмотреть это фильм бесплатно так , что не делайте ошибки и не платите за этот треш. Это фильм достоин только места в худшей сотке.


? i was one chosen to see this movie in a sneak preview br br first you should know that this film is based on the video game far cry a for its time r
eally good game 2004 second you should know that the of this flick is the great uwe boll this is a man who takes video games ? siege ? ? etc and makes
 movies out of them very horrible ones br br i still remember when i saw ? the king's swords a ? siege tail there were so horrible mistakes in this fi
lm like 3 scenes playing at the same time 2 at day time and one somehow at night br br so lets come to far cry if you expect cool action forget it rea
lly cheap tricks and a plastic helicopter are far away from real action if you expect a cool story forget it by the not so bad story of the game this 
movie is a laugh the actors' playing makes the movie in a lot of moments funny but in a no good way br br i had the chance to see this movie for free 
so do not do the mistake and pay for this trash its one of my flicks for the bottom 100

негатив 3. test_data[24506]

? я люблю Стивена Сигала, но я не имею понятия о чем был это  фильм. Обычно меня не легко запутать сюжетом, но я так и не вьехал кто был на  чей стороне. Надеюсь его будущие фильмы будут немного лучше, тем более моя жена гооврит, что ему очень к лицу черный цвет ? всеже р

? i like steven seagal but i have not a clue what this movie was about i am not easily lost with movies but i had no idea who was on who's side br br 
hopefully some of his future movies will be a little better my wife still thinks he looks good in black ? however p

позитив 1. test_data[24603]

Это один из тех фильмов, которые я могу пересматривать раз за разом и не уставать от этого. До сего времени лучшая адаптация комедийной книги. Он мне нравится даже больше чем X-Men. По сути эта картина что то между иксменов и матрицы и она вышла еще и до того как появился герой Весли Снайпса Блэйд. Он просто не ? супергерой . Также  этот фильм не получил рейтниг PG-13. Конечно комиксы по большей части для детей, но мне больше нравится в омих фильмах. Давайте станем пред фактом лицом: если бы мы были в той же ситации стали ли бы ? шторм, так что это реалистичнее чем сам комикс. Адаптация также обладает кое чем, чего не имеют многие, а именно хорошей дракой в конце  между плохим и хорошим парнем. Давайте признаем, что ни один из фильмов про бетмена не имеет крутого боя в конце.

? this is one of those movies that i can watch again and again and not get tired of it it is by far one of the best comic book adaptations ever i like
d this one even more than x men in fact this movie is sort of a cross between x men and the matrix and it came out before either wesley snipes does a 
great job with the character of blade he is just not an ? super hero also this movie isn't to get a pg 13 rating sure comic books are for kids mainly 
but i like a little more in my movies let's face it if we were in these situations we would ? up a storm to so it is more realistic this comic book ad
aptation also has something that many don't a good fight in the end between the bad guy and good guy let's face it none of the batman movies had a ver
y good ending fight

позитив 2. test_data[24605]

? "Рей" один из тех фильмов котороый заставлет сделать паузу и задуматься над тем, что ты знаешь  об этом человеке на основе того что читал и слышал о нем и понимаешь, что это даже не сопоставимо. Во время моего первого просмотра я чувствовал себя как ? и забыл что смотрю фильм, а не наблюдаю за происходящим через свое окно. Эта картина настолько неотразима, она засасывает в себя, вовлекает в каждую деталь, ты попадаешь на поездкку по американской горке эмоций и когда все заканчивается ты не хочешь повторять опыт в скором времени, потому что это фильм имеет такой эмоциональный удар, которого многим не хватает. Jamie Foxx заслужил премию Оскар на всех основаниях. Его испольнение великолепно и должно считаться стандартом для любого кто желает сделать чьюто биографию ? в скором времени. Этот фильм настолько же хорошо как и обьект которого он освещает - оба класскика и легенда.

? ray is one of those movies that makes you pause you actually think about what you heard or think about what you read about this man and it doesn't e
ven come close during my first viewing of ray i forgot i was watching a movie i felt like a ? tom watching this man's life thru a window this movie is
 so compelling it drags you in and it involves your every emotion you go thru a emotional roller coaster ride and when it's over you don't want to do 
it again so soon because it has that kind of emotional punch that other movies are lacking jamie foxx deserved his oscar and quite rightfully so his p
erformance is spectacular and it should be held up as the standard for anybody wanting to do a bio ? anytime soon this movie is as good as it's subjec
t both deserved the titles classic and legend

позитив 3. test_data[2100]

? я был ярым ? фанатом с поздних 70-х и ранних 80-х. Когда  вышел фильм я решил его обязательно посмотреть и после просмотра пошел прямиком в магазин и купил его. Фильм прекрасен, я люблю его анимированные сцены и музыка шикарна. Да, я купил саундтрек на сиди. Рекомендую каждому получить удовольствие от этого фильма. 

? i have been an avid ? fan since the late 70's early 80's when this movie came out it was a must to see it and after seeing it i went right over and 
bought it the movie is great i love the animated action it brings and the music is great yes i bought the soundtrack on cd a recommended video for eve
ryone to watch and enjoy

In [0]:
'''This example demonstrates the use of fasttext for text classification

Based on Joulin et al's paper:

Bags of Tricks for Efficient Text Classification
https://arxiv.org/abs/1607.01759

Results on IMDB datasets with uni and bi-gram embeddings:
    Uni-gram: 0.8813 test accuracy after 5 epochs. 8s/epoch on i7 cpu.
    Bi-gram : 0.9056 test accuracy after 5 epochs. 2s/epoch on GTx 980M gpu.
'''

from __future__ import print_function
import numpy as np

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import GlobalAveragePooling1D
from keras.datasets import imdb


def create_ngram_set(input_list, ngram_value=2):
    """
    Extract a set of n-grams from a list of integers.

    >>> create_ngram_set([1, 4, 9, 4, 1, 4], ngram_value=2)
    {(4, 9), (4, 1), (1, 4), (9, 4)}

    >>> create_ngram_set([1, 4, 9, 4, 1, 4], ngram_value=3)
    [(1, 4, 9), (4, 9, 4), (9, 4, 1), (4, 1, 4)]
    """
    return set(zip(*[input_list[i:] for i in range(ngram_value)]))


def add_ngram(sequences, token_indice, ngram_range=2):
    """
    Augment the input list of list (sequences) by appending n-grams values.

    Example: adding bi-gram
    >>> sequences = [[1, 3, 4, 5], [1, 3, 7, 9, 2]]
    >>> token_indice = {(1, 3): 1337, (9, 2): 42, (4, 5): 2017}
    >>> add_ngram(sequences, token_indice, ngram_range=2)
    [[1, 3, 4, 5, 1337, 2017], [1, 3, 7, 9, 2, 1337, 42]]

    Example: adding tri-gram
    >>> sequences = [[1, 3, 4, 5], [1, 3, 7, 9, 2]]
    >>> token_indice = {(1, 3): 1337, (9, 2): 42, (4, 5): 2017, (7, 9, 2): 2018}
    >>> add_ngram(sequences, token_indice, ngram_range=3)
    [[1, 3, 4, 5, 1337, 2017], [1, 3, 7, 9, 2, 1337, 42, 2018]]
    """
    new_sequences = []
    for input_list in sequences:
        new_list = input_list[:]
        for ngram_value in range(2, ngram_range + 1):
            for i in range(len(new_list) - ngram_value + 1):
                ngram = tuple(new_list[i:i + ngram_value])
                if ngram in token_indice:
                    new_list.append(token_indice[ngram])
        new_sequences.append(new_list)

    return new_sequences

# Set parameters:
# ngram_range = 2 will add bi-grams features
ngram_range = 2
max_features = 20000
maxlen = 400
batch_size = 32
embedding_dims = 50
epochs = 5

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')
print('Average train sequence length: {}'.format(np.mean(list(map(len, x_train)), dtype=int)))
print('Average test sequence length: {}'.format(np.mean(list(map(len, x_test)), dtype=int)))

if ngram_range > 1:
    print('Adding {}-gram features'.format(ngram_range))
    # Create set of unique n-gram from the training set.
    ngram_set = set()
    for input_list in x_train:
        for i in range(2, ngram_range + 1):
            set_of_ngram = create_ngram_set(input_list, ngram_value=i)
            ngram_set.update(set_of_ngram)

    # Dictionary mapping n-gram token to a unique integer.
    # Integer values are greater than max_features in order
    # to avoid collision with existing features.
    start_index = max_features + 1
    token_indice = {v: k + start_index for k, v in enumerate(ngram_set)}
    indice_token = {token_indice[k]: k for k in token_indice}

    # max_features is the highest integer that could be found in the dataset.
    max_features = np.max(list(indice_token.keys())) + 1

    # Augmenting x_train and x_test with n-grams features
    x_train = add_ngram(x_train, token_indice, ngram_range)
    x_test = add_ngram(x_test, token_indice, ngram_range)
    print('Average train sequence length: {}'.format(np.mean(list(map(len, x_train)), dtype=int)))
    print('Average test sequence length: {}'.format(np.mean(list(map(len, x_test)), dtype=int)))

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen))

# we add a GlobalAveragePooling1D, which will average the embeddings
# of all words in the document
model.add(GlobalAveragePooling1D())

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))

Loading data...
25000 train sequences
25000 test sequences
Average train sequence length: 238
Average test sequence length: 230
Adding 2-gram features
Average train sequence length: 476
Average test sequence length: 428
Pad sequences (samples x time)
x_train shape: (25000, 400)
x_test shape: (25000, 400)
Build model...
Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 60s 2ms/step - loss: 0.5862 - acc: 0.7874 - val_loss: 0.4372 - val_acc: 0.8548
Epoch 2/5
25000/25000 [==============================] - 58s 2ms/step - loss: 0.2877 - acc: 0.9276 - val_loss: 0.3030 - val_acc: 0.8904
Epoch 3/5
25000/25000 [==============================] - 58s 2ms/step - loss: 0.1432 - acc: 0.9696 - val_loss: 0.2628 - val_acc: 0.8998
Epoch 4/5
25000/25000 [==============================] - 58s 2ms/step - loss: 0.0775 - acc: 0.9871 - val_loss: 0.2441 - val_acc: 0.9030
Epoch 5/5
25000/25000 [==============================] - 58s 2ms/step - loss: 0.0437 

In [0]:
import numpy as np
import matplotlib.pyplot as plt

from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Embedding, Reshape, Flatten
from keras import optimizers

from keras.models import Model
from keras.layers import concatenate, Input

num_features = 3000
sequence_length = 300
embedding_dimension = 100

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = num_features)

X_train = pad_sequences(X_train, maxlen = sequence_length)
X_test = pad_sequences(X_test, maxlen = sequence_length)

filter_sizes = [3, 4, 5]

def convolution():
    inn = Input(shape = (sequence_length, embedding_dimension, 1))
    convolutions = []
    # we conduct three convolutions & poolings then concatenate them.
    for fs in filter_sizes:
        conv = Conv2D(filters = 100, kernel_size = (fs, embedding_dimension), strides = 1, padding = "valid")(inn)
        nonlinearity = Activation('relu')(conv)
        maxpool = MaxPooling2D(pool_size = (sequence_length - fs + 1, 1), padding = "valid")(nonlinearity)
        convolutions.append(maxpool)
        
    outt = concatenate(convolutions)
    model = Model(inputs = inn, outputs = outt)
        
    return model

def imdb_cnn_3():
    
    model = Sequential()
    model.add(Embedding(input_dim = 3000, output_dim = embedding_dimension, input_length = sequence_length))
    model.add(Reshape((sequence_length, embedding_dimension, 1), input_shape = (sequence_length, embedding_dimension)))
    
    # call convolution method defined above
    model.add(convolution())
    
    model.add(Flatten())
    model.add(Dense(10))
    model.add(Activation('relu'))
    model.add(Dropout(0.3))
    model.add(Dense(10))
    model.add(Activation('relu'))
    model.add(Dropout(0.3))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    adam = optimizers.Adam(lr = 0.001)

    model.compile(loss='binary_crossentropy', optimizer=adam , metrics=['accuracy'])
    
    return model

model = imdb_cnn_3()

history = model.fit(X_train, y_train, batch_size = 50, epochs = 100, validation_split = 0.2, verbose = 0)

# plt.plot(history.history['acc'])
# plt.plot(history.history['val_acc'])
# plt.legend(['training', 'validation'], loc = 'upper left')
# plt.show()

results = model.evaluate(X_test, y_test)
print('Test accuracy: ', results[1])

25000/25000 [==============================] - 7s 295us/step
Test accuracy:  0.87504


In [0]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.datasets import imdb

model_git = Sequential()

max_features = 5000
maxlen = 400
batch_size = 32
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 2



# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model_git.add(Embedding(5000,
                    50,
                    input_length=400))
model_git.add(Dropout(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model_git.add(Conv1D(250,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
# we use max pooling:
model_git.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model_git.add(Dense(250))
model_git.add(Dropout(0.2))
model_git.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model_git.add(Dense(1))
model_git.add(Activation('sigmoid'))

model_git.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

x_train = sequence.pad_sequences(train_data_imdb, maxlen=maxlen)
x_test = sequence.pad_sequences(test_data_imdb, maxlen=maxlen)

model_git.fit(x_train, train_labels_imdb,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, test_labels_imdb))

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 13s 521us/step - loss: 5.5483 - acc: 0.5672 - val_loss: 6.7034 - val_acc: 0.5692
Epoch 2/2
25000/25000 [==============================] - 12s 471us/step - loss: 6.9262 - acc: 0.5571 - val_loss: 6.3497 - val_acc: 0.5934


In [0]:
'''This example demonstrates the use of fasttext for text classification

Based on Joulin et al's paper:

Bags of Tricks for Efficient Text Classification
https://arxiv.org/abs/1607.01759

Results on IMDB datasets with uni and bi-gram embeddings:
    Uni-gram: 0.8813 test accuracy after 5 epochs. 8s/epoch on i7 cpu.
    Bi-gram : 0.9056 test accuracy after 5 epochs. 2s/epoch on GTx 980M gpu.
'''

from __future__ import print_function
import numpy as np

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import GlobalAveragePooling1D
from keras.datasets import imdb


def create_ngram_set(input_list, ngram_value=2):
    """
    Extract a set of n-grams from a list of integers.

    >>> create_ngram_set([1, 4, 9, 4, 1, 4], ngram_value=2)
    {(4, 9), (4, 1), (1, 4), (9, 4)}

    >>> create_ngram_set([1, 4, 9, 4, 1, 4], ngram_value=3)
    [(1, 4, 9), (4, 9, 4), (9, 4, 1), (4, 1, 4)]
    """
    return set(zip(*[input_list[i:] for i in range(ngram_value)]))


def add_ngram(sequences, token_indice, ngram_range=2):
    """
    Augment the input list of list (sequences) by appending n-grams values.

    Example: adding bi-gram
    >>> sequences = [[1, 3, 4, 5], [1, 3, 7, 9, 2]]
    >>> token_indice = {(1, 3): 1337, (9, 2): 42, (4, 5): 2017}
    >>> add_ngram(sequences, token_indice, ngram_range=2)
    [[1, 3, 4, 5, 1337, 2017], [1, 3, 7, 9, 2, 1337, 42]]

    Example: adding tri-gram
    >>> sequences = [[1, 3, 4, 5], [1, 3, 7, 9, 2]]
    >>> token_indice = {(1, 3): 1337, (9, 2): 42, (4, 5): 2017, (7, 9, 2): 2018}
    >>> add_ngram(sequences, token_indice, ngram_range=3)
    [[1, 3, 4, 5, 1337, 2017], [1, 3, 7, 9, 2, 1337, 42, 2018]]
    """
    new_sequences = []
    for input_list in sequences:
        new_list = input_list[:]
        for ngram_value in range(2, ngram_range + 1):
            for i in range(len(new_list) - ngram_value + 1):
                ngram = tuple(new_list[i:i + ngram_value])
                if ngram in token_indice:
                    new_list.append(token_indice[ngram])
        new_sequences.append(new_list)

    return new_sequences

# Set parameters:
# ngram_range = 2 will add bi-grams features
ngram_range = 2
max_features = 20000
maxlen = 400
batch_size = 32
embedding_dims = 50
epochs = 5

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')
print('Average train sequence length: {}'.format(np.mean(list(map(len, x_train)), dtype=int)))
print('Average test sequence length: {}'.format(np.mean(list(map(len, x_test)), dtype=int)))

if ngram_range > 1:
    print('Adding {}-gram features'.format(ngram_range))
    # Create set of unique n-gram from the training set.
    ngram_set = set()
    for input_list in x_train:
        for i in range(2, ngram_range + 1):
            set_of_ngram = create_ngram_set(input_list, ngram_value=i)
            ngram_set.update(set_of_ngram)

    # Dictionary mapping n-gram token to a unique integer.
    # Integer values are greater than max_features in order
    # to avoid collision with existing features.
    start_index = max_features + 1
    token_indice = {v: k + start_index for k, v in enumerate(ngram_set)}
    indice_token = {token_indice[k]: k for k in token_indice}

    # max_features is the highest integer that could be found in the dataset.
    max_features = np.max(list(indice_token.keys())) + 1

    # Augmenting x_train and x_test with n-grams features
    x_train = add_ngram(x_train, token_indice, ngram_range)
    x_test = add_ngram(x_test, token_indice, ngram_range)
    print('Average train sequence length: {}'.format(np.mean(list(map(len, x_train)), dtype=int)))
    print('Average test sequence length: {}'.format(np.mean(list(map(len, x_test)), dtype=int)))

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen))

# we add a GlobalAveragePooling1D, which will average the embeddings
# of all words in the document
model.add(GlobalAveragePooling1D())

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))

Loading data...
25000 train sequences
25000 test sequences
Average train sequence length: 238
Average test sequence length: 230
Adding 2-gram features
Average train sequence length: 476
Average test sequence length: 428
Pad sequences (samples x time)
x_train shape: (25000, 400)
x_test shape: (25000, 400)
Build model...
Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 60s 2ms/step - loss: 0.5862 - acc: 0.7874 - val_loss: 0.4372 - val_acc: 0.8548
Epoch 2/5
25000/25000 [==============================] - 58s 2ms/step - loss: 0.2877 - acc: 0.9276 - val_loss: 0.3030 - val_acc: 0.8904
Epoch 3/5
25000/25000 [==============================] - 58s 2ms/step - loss: 0.1432 - acc: 0.9696 - val_loss: 0.2628 - val_acc: 0.8998
Epoch 4/5
25000/25000 [==============================] - 58s 2ms/step - loss: 0.0775 - acc: 0.9871 - val_loss: 0.2441 - val_acc: 0.9030
Epoch 5/5
25000/25000 [==============================] - 58s 2ms/step - loss: 0.0437 